# Dataset Preparation


Here we will perform the **Preprocessing** of the dataset in a way that prevents data leakage and avoids inconsistencies. We will then split the dataset into a training part and a testing part.

## Load the Dataset

In [1]:
import pandas as pd 

from CogniPredictAD.preprocessing import ADNICleaner, ADNITransformator
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 116)
pd.set_option('display.max_columns', 116)
pd.set_option('display.max_info_columns', 116) 

new_dataset = pd.read_csv("../data/ADNIMERGE.csv")
new_dataset

C:\Users\Utente\AppData\Local\Temp\ipykernel_8248\2867084769.py:10: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  new_dataset = pd.read_csv("../data/ADNIMERGE.csv")


,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,PTETHCAT,PTRACCAT,PTMARRY,APOE4,FDG,PIB,AV45,FBB,ABETA,TAU,PTAU,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,DIGITSCOR,TRABSCOR,FAQ,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,FLDSTRENG,FSVERSION,IMAGEUID,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,DX,mPACCdigit,mPACCtrailsB,EXAMDATE_bl,CDRSB_bl,ADAS11_bl,ADAS13_bl,ADASQ4_bl,MMSE_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,LDELTOTAL_BL,DIGITSCOR_bl,TRABSCOR_bl,FAQ_bl,mPACCdigit_bl,mPACCtrailsB_bl,FLDSTRENG_bl,FSVERSION_bl,IMAGEUID_bl,Ventricles_bl,Hippocampus_bl,WholeBrain_bl,Entorhinal_bl,Fusiform_bl,MidTemp_bl,ICV_bl,MOCA_bl,EcogPtMem_bl,EcogPtLang_bl,EcogPtVisspat_bl,EcogPtPlan_bl,EcogPtOrgan_bl,EcogPtDivatt_bl,EcogPtTotal_bl,EcogSPMem_bl,EcogSPLang_bl,EcogSPVisspat_bl,EcogSPPlan_bl,EcogSPOrgan_bl,EcogSPDivatt_bl,EcogSPTotal_bl,ABETA_bl,TAU_bl,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,16,Not Hisp/Latino,White,Married,0.0,1.33615,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.67,18.67,5.0,28.0,44.0,4.0,6.0,54.5455,10.0,34.0,112.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35475.0,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,CN,-4.31028,-4.11443,2005-09-08,0.0,10.67,18.67,5.0,28.0,44.0,4.0,6.0,54.5455,10.0,34.0,112.0,0.0,-4.31028,-4.114430,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35475.0,118233.0,8336.0,1229740.0,4177.0,16559.0,27936.0,1984660.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.33615,NaN,NaN,NaN,0.000000,0.00000,0,0,2023-07-07 04:59:40.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.10860,NaN,NaN,NaN,741.5,239.7,22.83,4.5,22.00,31.00,8.0,20.0,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32237.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,Dementia,-16.24490,-15.82990,2005-09-12,4.5,22.00,31.00,8.0,20.0,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,-16.24490,-15.829900,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32237.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.5,239.7,22.83,1.10860,NaN,NaN,NaN,0.000000,0.00000,0,0,2023-07-07 04:59:40.0
2,3,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.07796,NaN,NaN,NaN,NaN,NaN,NaN,6.0,19.00,30.00,10.0,24.0,19.0,2.0,6.0,100.0000,NaN,19.0,135.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,31863.0,88580.0,5446.0,1100060.0,2427.0,14400.0,16972.0,1906430.0,Dementia,-14.76900,-13.13950,2005-09-12,4.5,22.00,31.00,8.0,20.0,22.0,1.0,4.0,100.0000,2.0,25.0,148.0,10.0,-16.24490,-15.829900,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,32237.0,84599.0,5319.0,1129830.0,1791.0,15506.0,18422.0,1920690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,741.5,239.7,22.83,1.10860,NaN,NaN,NaN,0.498289,5.96721,6,6,2023-07-07 04:59:40.0
3,3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male,18,Not Hisp/Latino,White,Married,1.0,1.11626,NaN,NaN,NaN,601.4,251.7,24.18,3.5,24.00,35.00,10.0,17.0,31.0,2.0,7.0,100.0000,0.0,21.0,126.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5 Tesla MRI,Cross-Sectional FreeSurfer (FreeSurfer Version...,35576.0,90099.0,5157.0,1095640.0,1596.0,14617.0,17330.0,1903820.0,Dementia,-20.96910,-19.77900,2005-09-12,4.5,22.00,31.

## Data Cleaning

**Data Cleaning** is the first step of Preprocessing and it is used for detecting and correcting or removing errors and inconsistencies in the dataset to improve data quality and model performance. Clean data ensures that the model learns from accurate, relevant, and reliable inputs. 

In [2]:
cleaner = ADNICleaner(new_dataset)

**ADNICleaner** is a utility class for *Data Cleaning* and *Preprocessing* of the ***ADNIMERGE*** dataset. It stores a DataFrame and provides a collection of data-cleaning utilities tailored for baseline visit selection, missing-value handling, column consolidation, and dataset simplification

### Retain Baseline visits only

The goal of this project is to predict, using features derived at baseline, the diagnosis upon their last visit to a testing site. Therefore we do not need all visits, but simply the baseline visits and taking into account only the baseline visit indicators. Now we processe the ADNIMERGE dataset to retain only the baseline visit (based on `VISCODE`) for each patient (`PTID`). 

In [3]:
new_dataset = cleaner.select_baseline_visits()

The dataset has been filtered. Only valid baseline visits with non-null 'DX_bl' per patient are retained.
Rows before filtering:	 16421
Rows after filtering:	 2419
Rows removed:		 14002


We control if "_bl" values are NaN and their counter part are not NaN. 

In [4]:
new_dataset = cleaner.fill_missing_bl_from_original()

No missing '_bl' values were filled from base columns.


**Significant Memory Concern** (**SMC**) **is not a certified medical diagnosis**. Therefore, we want to replace it in `DX_bl` with a different diagnosis from the `DX` column. The following code does exactly that. It takes all the rows with the diagnosis "SMC" and attempts to replace them using information from the complementary dataset. If the closest visit that has a diagnosis has:
- **CN** → **it becomes CN**
- **MCI** → **it becomes EMCI**
- if they have **Dementia** → **it becomes AD**
- if there is no useful information → **by default "SMC" becomes CN**
Finally, it prints how many replacements were made for each case.

In [5]:
new_dataset = cleaner.remap_smc_baseline()

SMC -> CN (direct from DX): 353
SMC -> EMCI (direct from DX MCI): 0
SMC -> AD (direct from DX Dementia): 0
SMC -> CN (fallback for leftovers): 0
TOTAL SMC replacements (all targets): 353


`DX_bl` indicates the screening diagnosis, i.e. the preliminary clinical judgment assigned during the first evaluation visit. It is a Screening diagnosis.

`DX` is instead the diagnosis assigned during the baseline visit (denoted by `VISCODE` == "bl"), after a more in-depth clinical evaluation. 

Let's compare their values.

In [6]:
counts = (
    new_dataset[new_dataset["DX"] == "CN"]["DX_bl"]
    .value_counts()
)

print(counts)

counts = (
    new_dataset[new_dataset["DX"] == "MCI"]["DX_bl"]
    .value_counts()
)

print("\n")
print(counts)

counts = (
    new_dataset[new_dataset["DX"] == "Dementia"]["DX_bl"]
    .value_counts()
)

print("\n")
print(counts)

DX_bl
CN      893
EMCI      2
Name: count, dtype: int64


DX_bl
LMCI    683
EMCI    417
AD        1
Name: count, dtype: int64


DX_bl
AD      407
LMCI      6
Name: count, dtype: int64


**These discrepancies do not indicate input or transcription errors, but reflect actual changes in diagnosis, determined by a more complete clinical evaluation at the baseline visit.** However, we want a more precise diagnosis. We assume that if `DX` is **MCI**, `DX_bl` correctly separates between **EMCI** and **LMCI**. We attempt **EMCI** and **LMCI** because they are well-recognized diagnostic classes that have different magnitudes, even though they are diagnostic subclasses of **MCI**. This provides a more complete prediction of the type of MCI. We replace the remaining values ​​with the correct diagnosis.

In [7]:
new_dataset = cleaner.fix_dx_bl_discrepancies()

Fixed DX_bl discrepancies:
 - DX='CN' & DX_bl='EMCI'  -> DX_bl='CN'   : 2 rows changed
 - DX='MCI' & DX_bl='AD'   -> DX_bl='LMCI' : 1 rows changed
 - DX='Dementia' & DX_bl='LMCI' -> DX_bl='AD' : 6 rows changed
TOTAL rows modified: 9


We simplify the dataset by handling columns that have both a regular and a _bl (baseline) version. 

In [8]:
new_dataset = cleaner.consolidate_bl_columns()

Dropped 50 columns: ['DX', 'EXAMDATE', 'CDRSB', 'ADAS11', 'ADAS13', 'ADASQ4', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning', 'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'LDELTOTAL', 'DIGITSCOR', 'TRABSCOR', 'FAQ', 'mPACCdigit', 'mPACCtrailsB', 'FLDSTRENG', 'FSVERSION', 'IMAGEUID', 'Ventricles', 'Hippocampus', 'WholeBrain', 'Entorhinal', 'Fusiform', 'MidTemp', 'ICV', 'MOCA', 'EcogPtMem', 'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan', 'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang', 'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt', 'EcogSPTotal', 'ABETA', 'TAU', 'PTAU', 'FDG', 'PIB', 'AV45', 'FBB', 'Month']
Renamed 51 columns.


We now remove single value attributes. 

In [9]:
new_dataset = cleaner.remove_single_value_attributes()

The following single-value columns have been dropped:
['VISCODE', 'Years', 'Month', 'M']


We remove duplicated rows. 

In [10]:
new_dataset = cleaner.remove_duplicate_rows()

No duplicate rows found.


### Handling Missing Values

Calculates the percentage of having NULL values in each column and prints in descending order.

In [11]:
print("Percentage of NULL values per column:")
null_percent = (new_dataset.isna().sum() / len(new_dataset['RID']) * 100).sort_values(ascending=False)
print(null_percent.apply(lambda x: f"{x:.2f}%"))

Percentage of NULL values per column:
PIB                      99.17%
FBB                      85.78%
DIGITSCOR                66.35%
AV45                     53.20%
ABETA                    49.77%
TAU                      49.77%
PTAU                     49.77%
EcogSPOrgan              37.83%
FDG                      37.45%
EcogSPDivatt             36.30%
EcogSPVisspat            36.13%
EcogPtOrgan              35.63%
EcogSPPlan               35.59%
MOCA                     35.26%
EcogSPMem                35.14%
EcogSPTotal              35.14%
FLDSTRENG                35.14%
EcogSPLang               35.06%
EcogPtVisspat            34.68%
EcogPtDivatt             34.60%
EcogPtLang               34.31%
EcogPtMem                34.31%
EcogPtPlan               34.27%
EcogPtTotal              34.23%
MidTemp                  15.46%
Fusiform                 15.46%
Entorhinal               15.46%
Hippocampus              14.26%
APOE4                     8.64%
Ventricles                7.19%
Wh

We eliminate all columns that have missing values greater than 50%.

In [12]:
new_dataset = cleaner.remove_attributes_by_percentage(50)

The following columns have been dropped:
['PIB', 'FBB', 'DIGITSCOR', 'AV45']


We have `PTMARRY` that is a categorical attribute with only a missing value. 

In [13]:
print(new_dataset['PTMARRY'].value_counts())

PTMARRY
Married          1818
Widowed           268
Divorced          225
Never married      98
Unknown             9
Name: count, dtype: int64


Since there is already an "Unknown" value, we replace the NULL data in PTMARRY with "Unknown".

In [14]:
new_dataset = cleaner.fill_null_with_value("PTMARRY", "Unknown")

NaN values in column 'PTMARRY' have been replaced with 'Unknown'.


We want to replace some missing values. The code below creates a "*complementary*" dataset from the same CSV file (***ADNIMERGE***), filtering the initial rows (VISCODE == "bl") and removing columns corresponding to baseline values ​​(_bl). It converts the exam dates to datetime format and identifies the common columns between the two datasets, excluding the keys (PTID, DX) and EXAMDATE, which will be candidates for filling the missing values.

The code then organizes the complementary dataset by subject (PTID), sorting by exam date in descending order, and defines a mapping between the DX values ​​of the two datasets to consider clinical equivalences. Subsequently, for each row in the main dataset, it searches for matching rows in the complementary dataset with the same subject and a compatible DX, taking only consecutive blocks of recent rows with equivalent DX.

Finally, for each fillable column, if the value in the main row is missing, it replaces it with the first non-zero value found in the selected block of the complementary dataset, updating counters for the rows and cells actually filled. Finally, it prints how many values ​​were inserted.

In [ ]:
complementary = pd.read_csv("../data/ADNIMERGE.csv")

# Filter out rows where VISCODE is "bl"
complementary = complementary[complementary["VISCODE"] != "bl"].copy()

# Drop all columns that end with "_bl"
cols_to_drop = [col for col in complementary.columns if col.endswith("_bl")]
complementary.drop(columns=cols_to_drop, inplace=True)

# Ensure EXAMDATE columns are parsed as datetime if present
for df_name, df in [("new_dataset", new_dataset), ("complementary", complementary)]:
    if "EXAMDATE" in df.columns:
        df["EXAMDATE"] = pd.to_datetime(df["EXAMDATE"], errors="coerce")

# Identify common columns 
common_cols = [c for c in new_dataset.columns if c in complementary.columns]

# Required key columns
key_cols = {"PTID", "DX"}
if not key_cols.issubset(set(new_dataset.columns)) or not key_cols.issubset(set(complementary.columns)):
    raise ValueError("Both datasets must contain 'PTID' and 'DX' columns.")

# Columns eligible for filling (exclude the keys and EXAMDATE)
fill_cols = [c for c in common_cols if c not in ("PTID", "DX", "EXAMDATE")]

# Prepare complementary grouped and sorted by EXAMDATE desc for faster retrieval per PTID
# If EXAMDATE is missing in complementary, we keep original order (no sort).
if "EXAMDATE" in complementary.columns:
    complementary_sorted = complementary.sort_values(["PTID", "EXAMDATE"], ascending=[True, False]).copy()
else:
    complementary_sorted = complementary.copy()

# Build a dict: pid -> sub-DataFrame (sorted desc by EXAMDATE if available)
comp_by_pid = {pid: grp for pid, grp in complementary_sorted.groupby("PTID")}

# --- NEW: DX equivalence mapping ---
# new_dataset DX: "CN", "EMCI", "LMCI", "AD"
# complementary DX: "CN", "MCI", "Dementia"
dx_equivalence = {
    "CN":     ["CN"],
    "EMCI":   ["MCI"],
    "LMCI":   ["MCI"],
    "AD":     ["Dementia"],
}

def complementary_dx_matches(new_dx, comp_dx):
    """
    Return True if comp_dx is considered matching new_dx according to the mapping rules.
    If new_dx is not in the mapping, fallback to exact equality check.
    """
    if pd.isna(new_dx) or pd.isna(comp_dx):
        return False
    allowed = dx_equivalence.get(new_dx)
    if allowed is None:
        # fallback: require exact match if new_dx not in expected mapping
        return comp_dx == new_dx
    return comp_dx in allowed

filled_cells = 0
filled_rows = set()

# Iterate over new_dataset rows and try to fill NaNs from complementary
for idx, row in new_dataset.iterrows():
    pid = row["PTID"]
    target_dx = row["DX"]
    # if PTID not in complementary --> nothing to do
    if pid not in comp_by_pid:
        continue

    comp_rows = comp_by_pid[pid]
    # If EXAMDATE exists in new row, keep only complementary rows that are >= new_row EXAMDATE (more recent or same date).
    # If new_row EXAMDATE is missing or complementary has no EXAMDATE, we keep all comp_rows (already sorted descending if EXAMDATE exists).
    if "EXAMDATE" in new_dataset.columns and "EXAMDATE" in comp_rows.columns:
        new_exam = row.get("EXAMDATE", pd.NaT)
        if pd.notna(new_exam):
            comp_rows = comp_rows[comp_rows["EXAMDATE"] >= new_exam]
            if comp_rows.empty:
                continue

    # Now comp_rows is sorted with most recent first (if EXAMDATE existed). We must take the contiguous block
    # from the top where DX is considered matching based on the mapping. If top row has different (non-matching) DX, skip.
    comp_rows = comp_rows.copy()
    comp_rows.reset_index(drop=True, inplace=True)

    if comp_rows.shape[0] == 0:
        continue

    # Build boolean mask using the equivalence function
    # We vectorize by using the mapping list for the current target_dx when possible (faster)
    allowed_for_target = dx_equivalence.get(target_dx)
    if allowed_for_target is not None:
        mask_same_dx = comp_rows["DX"].isin(allowed_for_target)
    else:
        # fallback to exact equality if target_dx not in mapping
        mask_same_dx = comp_rows["DX"] == target_dx

    # If the most recent complementary visit does not match according to the rules, skip
    if not mask_same_dx.iloc[0]:
        continue

    # find the length of the initial contiguous True block (most recent block that matches)
    block_len = 0
    for val in mask_same_dx:
        if val:
            block_len += 1
        else:
            break

    if block_len == 0:
        continue

    block = comp_rows.iloc[:block_len]

    # For each fillable column, if new value is NaN, try to find the first non-null in block (most recent)
    any_filled_for_row = False
    for col in fill_cols:
        if col not in new_dataset.columns:
            continue

        if pd.isna(row[col]):
            # find first non-null in block
            non_null_series = block[col].dropna()
            if not non_null_series.empty:
                value_to_use = non_null_series.iloc[0]
                new_dataset.at[idx, col] = value_to_use
                filled_cells += 1
                any_filled_for_row = True

    if any_filled_for_row:
        filled_rows.add(idx)

print(f"Filling complete. {filled_cells} cells filled across {len(filled_rows)} rows.")


C:\Users\Utente\AppData\Local\Temp\ipykernel_8248\1301174851.py:1: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  complementary = pd.read_csv("../data/ADNIMERGE.csv")


Filling complete. 1578 cells filled across 282 rows.


In [16]:
print("Percentage of NULL values per column:")
null_percent = (new_dataset.isna().sum() / len(new_dataset['RID']) * 100).sort_values(ascending=False)
print(null_percent.apply(lambda x: f"{x:.2f}%"))

Percentage of NULL values per column:
ABETA                    49.44%
PTAU                     49.44%
TAU                      49.44%
FDG                      37.08%
EcogSPOrgan              33.94%
FLDSTRENG                33.11%
EcogSPDivatt             33.07%
EcogSPVisspat            32.99%
EcogPtOrgan              32.66%
EcogSPPlan               32.58%
EcogSPMem                32.41%
EcogSPTotal              32.37%
EcogSPLang               32.33%
MOCA                     32.24%
EcogPtVisspat            32.04%
EcogPtDivatt             31.91%
EcogPtPlan               31.67%
EcogPtLang               31.67%
EcogPtMem                31.62%
EcogPtTotal              31.62%
MidTemp                  12.73%
Fusiform                 12.73%
Entorhinal               12.73%
Hippocampus              10.71%
APOE4                     8.64%
Ventricles                5.70%
WholeBrain                4.30%
ICV                       2.89%
FSVERSION                 2.81%
IMAGEUID                  2.81%
TR

### Handling Dirty Values


We see categorical attributes via the *describe()* function.

In [17]:
new_dataset.describe(include=['O']).T 

,count,unique,top,freq
COLPROT,2419,4,ADNI1,819
ORIGPROT,2419,4,ADNI1,819
PTID,2419,2419,035_S_7121,1
DX,2419,4,CN,897
PTGENDER,2419,2,Male,1270
PTETHCAT,2419,3,Not Hisp/Latino,2291
PTRACCAT,2419,7,White,2135
PTMARRY,2419,5,Married,1818
FLDSTRENG,1618,2,1.5 Tesla MRI,818
FSVERSION,2351,3,Cross-Sectional FreeSurfer (5.1),915


`TAU`, `PTAU` e `ABETA` should be numerical values, so we check the range of values. 

In [18]:
print(new_dataset['TAU'].value_counts())
print(new_dataset['PTAU'].value_counts())
print(new_dataset['ABETA'].value_counts())

TAU
<80      4
247.4    3
154.1    3
157.3    3
168.3    3
        ..
168.6    1
239.6    1
215.5    1
185.4    1
224.4    1
Name: count, Length: 1119, dtype: int64
PTAU
<8       5
18.85    4
21.4     4
24.9     3
16.77    3
        ..
15.8     1
26.11    1
25.27    1
17.04    1
21.23    1
Name: count, Length: 1120, dtype: int64
ABETA
>1700     192
1435.0      3
1086.0      3
1245.0      3
1244.0      3
         ... 
314.7       1
390.9       1
529.3       1
428.8       1
690.2       1
Name: count, Length: 972, dtype: int64


Notice that there are "<" and ">" inside the numbers, which leads to some dirty data to deal with. So we clean the TAU, PTAU and ABETA columns in our dataset by converting values that include detection limit symbols ("<" or ">", like ">1700", "<8" and "<80") into numeric values. Values with < are slightly decreased using a scaling factor (default: 0.99) and values with > are slightly increased (default: 1.01). This helps make the data suitable for further analysis. 

In [19]:
new_dataset = cleaner.clean_limit_values(columns=["TAU", "PTAU", "ABETA"])
new_dataset[['TAU', 'PTAU', 'ABETA']].describe().T

Column 'TAU' cleaned: '<' values scaled by 0.99, '>' values by 1.01.
Column 'PTAU' cleaned: '<' values scaled by 0.99, '>' values by 1.01.
Column 'ABETA' cleaned: '<' values scaled by 0.99, '>' values by 1.01.


,count,mean,std,min,25%,50%,75%,max
TAU,1223.0,287.120450,132.868270,79.20,193.35,257.80,349.700,1313.0
PTAU,1223.0,27.644947,14.585780,7.92,17.35,24.13,33.975,121.2
ABETA,1223.0,982.730662,461.863177,198.00,595.45,854.20,1396.000,1717.0


Now we see `PTETHCAT` and `PTRACCAT` values.  

In [20]:
print(new_dataset['PTETHCAT'].value_counts())
print("\n")
print(new_dataset['PTRACCAT'].value_counts())

PTETHCAT
Not Hisp/Latino    2291
Hisp/Latino         116
Unknown              12
Name: count, dtype: int64


PTRACCAT
White                2135
Black                 177
Asian                  58
More than one          30
Unknown                12
Am Indian/Alaskan       5
Hawaiian/Other PI       2
Name: count, dtype: int64


We want to have much more readable and interpretable data. So we replace *Hisp/Latino* with *Hispanic* and *Not Hisp/Latino* with "Not Hispanic" inside `PTETHCAT`. We do the same thing with *Am Indian/Alaskan* replaced with *American Indian* and finally with *Hawaiian/Other PI* with *Hawaiian* in `PTRACCAT`.

In [21]:
new_dataset = cleaner.replace_value_in_column("PTETHCAT", "Hisp/Latino", "Hispanic")
new_dataset = cleaner.replace_value_in_column("PTETHCAT", "Not Hisp/Latino", "Not Hispanic")
new_dataset = cleaner.replace_value_in_column("PTRACCAT", "Am Indian/Alaskan", "American Indian")
new_dataset = cleaner.replace_value_in_column("PTRACCAT", "Hawaiian/Other PI", "Hawaiian")

Replaced all occurrences of 'Hisp/Latino' with 'Hispanic' in column 'PTETHCAT'.
Replaced all occurrences of 'Not Hisp/Latino' with 'Not Hispanic' in column 'PTETHCAT'.
Replaced all occurrences of 'Am Indian/Alaskan' with 'American Indian' in column 'PTRACCAT'.
Replaced all occurrences of 'Hawaiian/Other PI' with 'Hawaiian' in column 'PTRACCAT'.


## Data Transformation

**Data Transformation** is another step of Preprocessing. In Data Transformation, the data are transformed or consolidated into forms appropriate for mining. Strategies for data transformation include Normalization and Discretization. 

In [22]:
transformer = ADNITransformator(new_dataset)

**ADNITransformer** is a utility class for preprocessing the ***ADNIMERGE*** dataset. It stores a DataFrame and provides utilities to create ratio columns, normalize numeric features (Min-Max, Z-score, Robust), encode categorical variables (label, one-hot, ordinal), and perform different types of binning (equal-width, equal-frequency, custom). Each method updates and returns the internal dataset and prints a short summary of the action.

### Data Encoding


We start with the encoding of categorical value. 

In [23]:
new_dataset.describe(include=['O']).T  # describe for categorical variables

,count,unique,top,freq
COLPROT,2419,4,ADNI1,819
ORIGPROT,2419,4,ADNI1,819
PTID,2419,2419,035_S_7121,1
DX,2419,4,CN,897
PTGENDER,2419,2,Male,1270
PTETHCAT,2419,3,Not Hispanic,2291
PTRACCAT,2419,7,White,2135
PTMARRY,2419,5,Married,1818
FLDSTRENG,1618,2,1.5 Tesla MRI,818
FSVERSION,2351,3,Cross-Sectional FreeSurfer (5.1),915


The first one we consider is `PTETHCAT`. 

In [24]:
print(new_dataset['PTETHCAT'].value_counts())


PTETHCAT
Not Hispanic    2291
Hispanic         116
Unknown           12
Name: count, dtype: int64


We have chosen to apply One-Hot Encoding to represent this attribute. After that, we decide that `PTETHCAT` we will retain only the *Hispanic* category as a binary indicator. The *Not Hispanic* category is its logical complement, and therefore the information is preserved without explicitly encoding it. The *Unknown* category will be excluded from the encoding, as it is not relevant for our analysis.


In [25]:
new_dataset = transformer.one_hot_encoding(columns=["PTETHCAT"])
new_dataset.drop(["PTETHCAT_Not Hispanic"], axis=1, inplace=True)
new_dataset.drop(["PTETHCAT_Unknown"], axis=1, inplace=True)
new_dataset.rename(columns={'PTETHCAT_Hispanic': 'HISPANIC'}, inplace=True)
new_dataset['HISPANIC'] = new_dataset['HISPANIC'].astype(int)

One-Hot Encoding applied to columns: ['PTETHCAT']


For `PTRACCAT` instead we do not want to have 7 mutually exclusive Boolean columns and prefer to use Label Encoding. Reading the 2020 study "[Race, Ethnicity, and Alzheimer's,](https://aaic.alz.org/downloads2020/2020_Race_and_Ethnicity_Fact_Sheet.pdf)", compiled by the Alzheimer's Association, we realize that black people have twice the risk of developing Alzheimer's disease compared to white people, and that Hispanics have about 1.5 times the risk compared to non-Hispanic whites. 

In [26]:
print(new_dataset['PTRACCAT'].value_counts())
print("\n")

hispanic_subset = new_dataset[new_dataset['HISPANIC'] == 1]

# Count how many Hispanics of each "skin color" 
count_hispanic_by_race = hispanic_subset['PTRACCAT'].value_counts()

print("Number of Hispanics by PTRACCAT:")
print(count_hispanic_by_race)

PTRACCAT
White              2135
Black               177
Asian                58
More than one        30
Unknown              12
American Indian       5
Hawaiian              2
Name: count, dtype: int64


Number of Hispanics by PTRACCAT:
PTRACCAT
White            90
More than one    13
Unknown          12
Black             1
Name: count, dtype: int64


We see that the 12 *Unknown* values are also *Hispanic*, and that we can consider *Hispanic Whites* as *Hispanics* if we want. So we can merge the columns into a single column called "`PTETHNICITY`", keeping all the information and removing the Unknown values.

In [27]:
# Function to create PTETHNICITY based on HISPANIC and PTRACCAT
def assign_ptethnicity(row):
    if row['HISPANIC'] == 1 and row['PTRACCAT'] in ['White', 'Unknown']:
        return 'Hispanic'
    else:
        return row['PTRACCAT']

# We apply the function to create the new column
new_dataset['PTETHNICITY'] = new_dataset.apply(assign_ptethnicity, axis=1)

new_dataset.drop(columns=['HISPANIC', 'PTRACCAT'], axis=1, inplace=True)

print(new_dataset['PTETHNICITY'].value_counts())


PTETHNICITY
White              2045
Black               177
Hispanic            102
Asian                58
More than one        30
American Indian       5
Hawaiian              2
Name: count, dtype: int64


Now we use the *Label Encoding* for `PTETHNICITY`. 

In [28]:
new_dataset = transformer.label_encoding(columns=["PTETHNICITY"])

Label Encoding applied to 'PTETHNICITY':
  'American Indian' -> 0
  'Asian' -> 1
  'Black' -> 2
  'Hawaiian' -> 3
  'Hispanic' -> 4
  'More than one' -> 5
  'White' -> 6


Let's see the distribution of values ​​for `PTMARRY`.

In [29]:
print(new_dataset['PTMARRY'].value_counts())

PTMARRY
Married          1818
Widowed           268
Divorced          225
Never married      98
Unknown            10
Name: count, dtype: int64


We see that the value *Married* represents about 75% of the sample. The other values ​​therefore don't have much relevance. We favor One Hot Encoding with a single Boolean `MARRIED`, which is 1 if the patient is married and 0 otherwise.

In [30]:
new_dataset = transformer.one_hot_encoding(columns=["PTMARRY"])
new_dataset.drop(["PTMARRY_Divorced"], axis=1, inplace=True)
new_dataset.drop(["PTMARRY_Never married"], axis=1, inplace=True)
new_dataset.drop(["PTMARRY_Unknown"], axis=1, inplace=True)
new_dataset.drop(["PTMARRY_Widowed"], axis=1, inplace=True)
new_dataset.rename(columns={'PTMARRY_Married': 'MARRIED'}, inplace=True)
new_dataset['MARRIED'] = new_dataset['MARRIED'].astype(int)

One-Hot Encoding applied to columns: ['PTMARRY']


For `PTGENDER` we simple render map *Male* as 1 and *Female* as 0. 

In [31]:
new_dataset['PTGENDER'] = new_dataset['PTGENDER'].map({'Male': 1, 'Female': 0})

Finally we map the diagnosis ***`DX`***, the class of our dataset, in an ordinal way.

In [32]:
mapping = {'DX': {'CN':0, 'EMCI':1, 'LMCI':2, 'AD':3}}

new_dataset = transformer.ordinal_encoding(
    columns=['DX'], 
    mapping=mapping,
)


Ordinal Encoding by mapping applied to 'DX': {'CN': 0, 'EMCI': 1, 'LMCI': 2, 'AD': 3}


## Data Reduction

**Data Reduction** is a preprocessing step in data analysis aimed at minimizing the volume of data while preserving its essential information and structure. It helps improve efficiency, reduce storage needs, and speed up processing by using techniques such as feature selection, dimensionality reduction, and data aggregation.

The source protocols (`COLPROT` and `ORIGPROT`) are not of interest to us for classifying the diagnosis of Alzheimer's disease. 

In [33]:
new_dataset.drop(["COLPROT", "ORIGPROT"], axis=1, inplace=True)

`SITE`, on the other hand, is the acquisition center (i.e., the facility, hospital, clinic, or laboratory) where the patient's data was collected. It is expressed in numerical format. This data, too, is not used to diagnose Alzheimer's disease based on baseline visit data.

In [34]:
new_dataset.drop(["SITE"], axis=1, inplace=True)

`EXAMDATE` is useless for our work. 

In [35]:
new_dataset.drop(["EXAMDATE"], axis=1, inplace=True)

It should be noted that ADNI not only contains tabular data, but also images that can be used for disease analysis, diagnosis, and prognosis, not to mention machine learning. Here, we are not interested in the images themselves, nor in any attributes related to how they were obtained and/or what they were obtained with.

`IMAGEUID` is the unique image identifier of the MRI scan. 

`FLDSTRENG` indicates the magnetic field strength of the MRI scanner used to acquire the image.

`FSVERSION` indicates the version of the FreeSurfer software used to process the MRI images.

It goes without saying that these two attributes are not relevant to our analysis, since we are attempting to perform an analysis of the baseline visits based on the values recorded by these analyses, without consulting the MRI. Therefore, we drop them immediately.

In [36]:
new_dataset.drop(["IMAGEUID", "FLDSTRENG", "FSVERSION"], axis=1, inplace=True)

`update_stamp` is a timestamp used to see the last update and we can also remove this for obvious reasons.

In [37]:
new_dataset.drop(["update_stamp"], axis=1, inplace=True)

Finally we delete the id columns: `RID` and `PTID`.

In [38]:
new_dataset.drop(["RID", "PTID"], axis=1, inplace=True)

## Error on RAVLT

We notice that there is an error in one of the RAVLT measurements. 

In [39]:
print(new_dataset.loc[new_dataset["RAVLT_perc_forgetting"] == -316.667,["RAVLT_perc_forgetting", "RAVLT_forgetting"]])

      RAVLT_perc_forgetting  RAVLT_forgetting
2285               -316.667             -19.0


It is easy to see that the value *-316.667* is almost certainly an **error** so we have a problem. -19 for `RAVLT forgetting` is off the scale as the score ranges from -15 to 15. We don't have the *trials* or the *delayed* scores, so we can't correctly estimate the two values. The best thing to do is delete the row.

In [40]:
new_dataset.drop(2285, inplace=True)
new_dataset.reset_index(drop=True, inplace=True)

## Data Splitting

**Data Splitting** is the division of the dataset into two parts:
- **Training set**: used to train the model.
- **Test set**: used to evaluate performance on previously unseen data.

The target class is ***`DX`*** (which was ***`DX_bl`***), but we're not separating it yet because we want to make some dataset considerations based on ***`DX`*** as well. We'll remove it in due course.

*We decide to split the data into 80% for the training set and 20% for the test set.*

In [ ]:
# Split directly on the whole dataset
train_set, test_set = train_test_split(
    new_dataset,
    test_size=0.2,                  # 20% of the data goes to testing
    random_state=42,
    stratify=new_dataset["DX"]      # Useful for classification: maintains class proportions. Stratify on the target column. 
)

train_set.to_csv('../data/pretrain.csv', index=False)
test_set.to_csv('../data/pretest.csv', index=False)
train_set

,DX,AGE,PTGENDER,PTEDUCAT,APOE4,CDRSB,ADAS11,ADAS13,ADASQ4,MMSE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,LDELTOTAL,TRABSCOR,FAQ,mPACCdigit,mPACCtrailsB,Ventricles,Hippocampus,WholeBrain,Entorhinal,Fusiform,MidTemp,ICV,MOCA,EcogPtMem,EcogPtLang,EcogPtVisspat,EcogPtPlan,EcogPtOrgan,EcogPtDivatt,EcogPtTotal,EcogSPMem,EcogSPLang,EcogSPVisspat,EcogSPPlan,EcogSPOrgan,EcogSPDivatt,EcogSPTotal,ABETA,TAU,PTAU,FDG,PTETHNICITY,MARRIED
0,2,77.1,0,16,1.0,2.5,3.00,5.00,1.0,28.0,47.0,5.0,7.0,63.63640,1.0,108.0,0.0,-4.84005,-4.499280,22503.0,5996.0,936074.0,2495.0,16048.0,14993.0,1325490.0,24.0,2.250,2.11111,1.00000,1.0,1.33333,1.00,1.56410,2.375,2.11111,2.42857,2.60,2.83333,2.75000,2.46154,744.4,407.1,30.40,1.22283,4,1
1,0,59.9,1,16,1.0,0.0,0.00,0.00,0.0,30.0,71.0,2.0,0.0,0.00000,19.0,47.0,0.0,5.42702,5.167270,43728.0,7600.0,1285280.0,4705.0,22083.0,24411.0,1707220.0,30.0,1.000,1.00000,1.00000,1.0,1.00000,1.00,1.00000,1.000,1.00000,1.00000,1.00,1.00000,1.00000,1.00000,772.8,166.6,15.80,1.16197,5,1
2,3,77.5,1,12,2.0,8.0,19.33,30.33,10.0,22.0,19.0,1.0,4.0,100.00000,0.0,300.0,25.0,-18.90540,-18.905400,109900.0,5812.0,1075140.0,2773.0,20675.0,19959.0,2057400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,393.8,212.4,18.56,NaN,6,1
3,2,82.1,1,20,0.0,1.5,13.00,21.00,8.0,26.0,35.0,1.0,6.0,85.71430,4.0,63.0,0.0,-7.95749,-7.967110,38990.0,6025.0,1028510.0,3360.0,21027.0,22485.0,1612770.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1717.0,396.2,34.68,1.11913,6,0
4,0,83.3,0,17,NaN,0.0,2.33,5.33,2.0,27.0,57.0,7.0,1.0,7.14286,13.0,98.0,3.0,-1.94841,-2.099850,73131.9,6461.1,890090.0,3040.0,15955.0,18179.0,1401100.0,25.0,1.250,1.33333,1.00000,1.0,1.33333,1.00,1.18421,1.375,1.11111,1.66667,1.00,1.83333,1.25000,1.36842,NaN,NaN,NaN,NaN,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1929,0,72.4,0,18,1.0,0.0,4.00,4.00,0.0,30.0,42.0,7.0,2.0,18.18180,11.0,52.0,0.0,2.22837,2.530720,42526.0,7716.0,1079110.0,5019.0,21334.0,20858.0,1519240.0,26.0,1.500,2.33333,1.28571,1.0,2.50000,1.25,1.71795,1.250,1.00000,1.20000,1.00,1.33333,1.50000,1.18919,1159.0,190.6,15.71,1.41610,6,0
1930,3,72.7,0,12,1.0,7.0,22.00,29.00,6.0,26.0,34.0,-1.0,6.0,100.00000,5.0,67.0,18.0,-9.28099,-6.698780,30697.0,5980.0,971114.0,2307.0,15802.0,15480.0,1364380.0,19.0,1.500,1.00000,1.14286,1.0,1.00000,1.00,1.12821,3.250,2.33333,2.42857,3.20,3.00000,3.50000,2.87179,568.8,430.1,46.04,1.26852,6,1
1931,0,70.5,0,17,0.0,0.0,19.00,23.00,4.0,29.0,31.0,4.0,3.0,42.85710,10.0,300.0,0.0,-2.30539,-5.033500,16297.0,6289.0,887360.0,3001.0,16867.0,16225.0,1247350.0,20.0,1.125,1.11111,1.00000,1.0,1.00000,1.00,1.05128,1.000,1.00000,1.00000,1.00,1.00000,1.00000,1.00000,1415.0,131.6,10.84,1.45617,6,1
1932,0,84.9,1,12,0.0,0.5,9.00,16.00,6.0,30.0,27.0,1.0,4.0,80.00000,13.0,65.0,0.0,-1.42719,-0.736856,39281.0,6693.0,972343.0,3189.0,18115.0,19954.0,1465660.0,26.0,2.000,2.00000,2.00000,2.0,1.50000,2.00,1.92105,1.625,1.22222,1.28571,1.25,1.00000,1.66667,1.32432,NaN,NaN,NaN,1.31888,6,0
